<a href="https://colab.research.google.com/github/Lavsdyc/Challenge-Telecom-X-an-lisis-de-evasi-n-de-clientes---Parte-2/blob/main/challenge_telecom_x_analisis_de_evasion_de_clientes_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>